<a href="https://colab.research.google.com/github/kriskirla/AllProjects/blob/master/Algotrading/Portfolio_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Description: Analyze my portfolio

In [ ]:
!pip install PyPortfolioOpt
!pip install pulp
!pip install yfinance

In [104]:
# Import libraries 
import pandas as pd
import numpy as np
import requests
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime as dt

In [105]:
# Tickers to track S&P500
payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = payload[0]
second_table = payload[1]

df = first_table
ticker = df['Symbol'].values.tolist()

In [106]:
# Get information about ticker
def get_company_information(symbol):
  ticker = yf.Ticker(symbol)
  return ticker.info

#get_company_information("ZOM")

In [136]:
# Get data from yahoo finance
df = yf.download(ticker, start="2015-01-01", end=dt.today().strftime('%Y-%m-%d'))['Close']
# Drop all delisted tickers
df = df.dropna(axis='columns', how='all')
df = df.drop(columns=["WRK"]) # For some reason WRK is showing Na for all except latest
df

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,...,VFC,VIAC,VLO,VMC,VNO,VNT,VRSK,VRSN,VRTX,VTR,VTRS,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,40.560001,53.910000,158.559998,27.332500,65.889999,90.459999,37.310001,44.900002,88.839996,72.339996,55.540001,52.020000,83.290001,59.529999,45.990002,61.150002,13.700000,30.540001,56.110001,67.769997,46.980000,63.250000,60.310001,56.200001,60.330002,70.120003,55.049999,186.600006,24.959999,11.20,2.670000,52.619999,159.889999,132.089996,99.669998,308.519989,62.680000,81.639999,125.459999,94.330002,...,69.453857,NaN,50.340000,66.980003,87.332130,NaN,63.970001,57.189999,123.070000,83.715881,56.349998,46.959999,87.110001,113.879997,76.000000,110.430000,53.189999,76.949997,54.700001,193.009995,118.013245,51.750000,45.220001,85.900002,33.766666,52.160000,17.930000,36.110001,146.889999,36.119999,43.599998,92.830002,51.930000,36.231884,38.080002,52.012939,112.589996,77.430000,28.290001,43.310001
2015-01-05,39.799999,53.880001,156.470001,26.562500,64.650002,89.690002,37.070000,44.910000,87.339996,71.980003,54.529999,50.220001,83.230003,58.660000,45.529999,60.250000,13.310000,29.740000,55.000000,66.250000,46.700001,61.939999,58.310001,56.990002,59.470001,68.730003,54.939999,182.169998,24.160000,11.20,2.660000,51.040001,157.990005,127.849998,98.230003,302.190002,63.799999,80.860001,123.260002,93.419998,...,68.870056,NaN,48.270000,65.129997,87.551620,NaN,63.139999,56.240002,121.029999,84.138405,55.720001,46.570000,85.139999,113.019997,74.500000,108.970001,52.540001,77.900002,53.200001,189.029999,117.165565,51.230000,43.430000,85.650002,33.466667,51.439999,17.760000,36.110001,144.389999,35.709999,42.799999,90.290001,51.570000,35.441368,35.709999,50.956146,116.790001,76.339996,27.230000,43.049999
2015-01-06,39.180000,53.040001,156.360001,26.565001,64.330002,90.180000,36.130001,44.400002,86.709999,70.529999,53.250000,49.230000,82.540001,57.500000,45.000000,60.580002,13.010000,29.475000,54.220001,65.330002,46.610001,60.669998,57.169998,57.450001,58.770000,68.680000,54.849998,177.949997,23.250000,11.20,2.630000,50.060001,152.899994,124.410004,97.970001,295.290009,63.810001,79.260002,124.940002,92.860001,...,68.465157,NaN,47.680000,64.980003,88.261276,NaN,62.919998,55.180000,117.360001,86.011192,54.180000,47.040001,84.040001,112.529999,74.690002,106.540001,52.680000,79.010002,52.090000,187.470001,116.582779,51.270000,42.840000,86.309998,33.380001,50.560001,17.730000,36.070000,141.309998,35.900002,42.180000,89.809998,50.930000,34.967064,35.500000,50.330696,115.800003,75.790001,26.190001,42.630001
2015-01-07,39.700001,53.009998,159.720001,26.937500,66.930000,91.980003,37.279999,44.759998,88.529999,71.110001,53.810001,49.970001,83.260002,57.380001,45.349998,61.439999,13.030000,29.660000,54.480000,65.589996,47.060001,60.160000,56.939999,59.570000,59.669998,69.809998,55.599998,187.929993,23.469999,10.72,2.580000,50.209999,158.240005,126.620003,99.000000,298.420013,65.139999,79.709999,125.559998,93.919998,...,69.340866,NaN,47.310001,66.209999,90.126877,NaN,62.959999,55.639999,120.660004,87.290169,56.110001,46.189999,84.860001,115.930000,76.599998,108.139999,53.330002,79.739998,52.400002,191.880005,118.251656,51.310001,42.610001,88.599998,33.413334,51.380001,17.559999,36.180000,145.460007,36.209999,42.200001,90.720001,52.250000,35.415020,35.779999,51.998562,118.680000,77.720001,26.44000

In [ ]:
# Calculate MacD, simply for displaying
exp1 = df.ewm(span=12, adjust=False).mean()
exp2 = df.ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
exp3 = macd.ewm(span=9, adjust=False).mean()
macd.plot(label='AAPL MACD', color='g')
ax = exp3.plot(label='Signal Line', color='r')
df.plot(ax=ax, secondary_y=True, label='AAPL')

The Parts above is to input your portfolio

The parts below is to optimize portfolio

In [ ]:
# Optimize the portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [137]:
# Calculate the expected annualized returns and the annualized sample covariance matrix of the daily asset returns
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimize for the maximal Sharpe ratio
ef = EfficientFrontier(mu, S) # Create the Efficient Frontier Object
weights = ef.max_sharpe()
clean_weights = ef.clean_weights()
print(clean_weights)
ef.portfolio_performance(verbose=True)

/usr/local/lib/python3.6/dist-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
/usr/local/lib/python3.6/dist-packages/pypfopt/risk_models.py:87: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn("Could not fix matrix. Please try a different risk model.")


OrderedDict([('A', 0.0), ('AAL', 0.0), ('AAP', 0.0), ('AAPL', 0.0), ('ABBV', 0.0), ('ABC', 0.0), ('ABMD', 0.0), ('ABT', 0.0), ('ACN', 0.0), ('ADBE', 0.0), ('ADI', 0.0), ('ADM', 0.0), ('ADP', 0.0), ('ADSK', 0.0), ('AEE', 0.0), ('AEP', 0.0), ('AES', 0.0), ('AFL', 0.0), ('AIG', 0.0), ('AIZ', 0.0), ('AJG', 0.0), ('AKAM', 0.0), ('ALB', 0.0), ('ALGN', 0.0), ('ALK', 0.0), ('ALL', 0.0), ('ALLE', 0.0), ('ALXN', 0.0), ('AMAT', 0.0), ('AMCR', 0.0), ('AMD', 0.05416), ('AME', 0.0), ('AMGN', 0.0), ('AMP', 0.0), ('AMT', 0.0), ('AMZN', 0.09703), ('ANET', 0.0), ('ANSS', 0.0), ('ANTM', 0.0), ('AON', 0.0), ('AOS', 0.0), ('APA', 0.0), ('APD', 0.0), ('APH', 0.0), ('APTV', 0.0), ('ARE', 0.0), ('ATO', 0.0), ('ATVI', 0.0), ('AVB', 0.0), ('AVGO', 0.0), ('AVY', 0.0), ('AWK', 0.0), ('AXP', 0.0), ('AZO', 0.0), ('BA', 0.0), ('BAC', 0.0), ('BAX', 0.0), ('BBY', 0.0), ('BDX', 0.0), ('BEN', 0.0), ('BIIB', 0.0), ('BIO', 0.0), ('BK', 0.0), ('BKNG', 0.0), ('BKR', 0.0), ('BLK', 0.0), ('BLL', 0.0), ('BMY', 0.0), ('BR', 0.0

(1.2082956751861982, 0.30693666489388055, 3.8714686484165592)

In [138]:
# Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

porfolio_val = 20000
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(clean_weights, latest_prices, porfolio_val)
allocation, leftover = da.lp_portfolio()
print(f"Discrete Allocation: {allocation}\nFunds Remaining: {leftover}")

Discrete Allocation: {'AMD': 11, 'AMZN': 1, 'CARR': 198, 'CLX': 9, 'DPZ': 6, 'NEM': 22, 'NFLX': 1, 'NVDA': 3, 'POOL': 2}
Funds Remaining: 0.40020751953125


In [139]:
# Save information to print
company_name = []
industry = []
share_price = []
total_price = []

for symbol in allocation:
    info = get_company_information(symbol)
    company_name.append(info['longName'])
    industry.append(info['industry'])
    share_price.append(info['regularMarketPrice'])
    total_price.append("${:.2f}".format(info['regularMarketPrice'] * allocation[symbol]))

In [140]:
# Print the final guestimation of what to buy
portfolio_df = pd.DataFrame(columns=["Ticker", "Name", "Allocation", "Price/Share", "Total", "Industry"])
portfolio_df["Ticker"] = allocation.keys()
portfolio_df["Name"] = company_name
portfolio_df["Allocation"] = allocation.values()
portfolio_df["Price/Share"] = share_price
portfolio_df["Total"] = total_price
portfolio_df["Industry"] = industry
portfolio_df

,Ticker,Name,Allocation,Price/Share,Total,Industry
0,AMD,"Advanced Micro Devices, Inc.",11,92.75,$1020.25,Semiconductors
1,AMZN,"Amazon.com, Inc.",1,3250.00,$3250.00,Internet Retail
2,CARR,Carrier Global Corporation,198,37.00,$7326.00,Building Products & Equipment
3,CLX,The Clorox Company,9,184.09,$1656.81,Household & Personal Products
4,DPZ,"Domino's Pizza, Inc.",6,385.07,$2310.42,Restaurants
5,NEM,Newmont Corporation,22,57.91,$1274.02,Gold
6,NFLX,"Netflix, Inc.",1,556.94,$556.94,Entertainment
7,NVDA,NVIDIA Corporation,3,602.00,$1806.00,Semiconductors
8,POOL,Pool Corporation,2,331.75,$663.50,Leisure
